# Usermore soap case study

Guilherme Fernandes e Lauro Solia

In [ ]:
from pprint import pprint

import gurobipy
from gurobipy import Model, GRB

import pandas as pd

## 1. Defining the sets

In [ ]:
plants = [
    "Covington",
    "New York",
    "Arlington",
    "Long Beach",
]

existing_warehouses = [
    "Atlanta",
    "Boston",
    "Buffalo",
    "Chicago",
    "Cleveland",
    "Davenport",
    "Detroit",
    "Grand Rapids",
    "Greensboro",
    "Kansas City",
    "Baltimore",
    "Memphis",
    "Milwaukee",
    "Orlando",
    "Pittsburgh",
    "Portland",
    "W Sacramento",
    "W Chester",
]

potential_warehouses = [
    "Albuquerque",
    "Billings",
    "Denver",
    "El Paso",
    "Camp Hill",
    "Houston",
    "Las Vegas",
    "Minneapolis",
    "New Orleans",
    "Phoenix",
    "Richmond",
    "St Louis",
    "Salt Lake City",
    "San Antonio",
    "Seattle",
    "Spokane",
    "San Francisco",
    "Indianapolis",
    "Louisville",
    "Columbus",
    "New York-",
    "Hartford",
    "Miami",
    "Mobile",
    "Memphis *",
    "Chicago *",
]

# Merge existing + potential
warehouses = plants + existing_warehouses  # + potential_warehouses

In [ ]:
print("Number of plants: ", len(plants))
print("Number of existing warehouses: ", len(existing_warehouses))
print("Number of potential warehouses: ", len(potential_warehouses))
print("Number of total warehouses: ", len(warehouses))
print("Obs.: Each plant is also an existing warehouse")

In [ ]:
# 90% under 1_500_000 miles
SERVICE_LEVEL_DISTANCE = 1000
SERVICE_LEVEL = 0.93

## 2. Parameters

In [ ]:
# read state demands from Figure 1
state_demand = {
    # West region:
    "WA": 32437,  # Washington
    "OR": 31365,  # Oregon
    "CA": 135_116,  # California
    "NV": 16755,  # Nevada
    "AZ": 9063,  # Arizona
    "ID": 7153,  # Idaho
    # Northwest region:
    "UT": 9001,  # Utah
    "MT": 4140,  # Montana
    "ND": 5703,  # North Dakota
    "WY": 1004,  # Wyoming
    "CO": 11147,  # Colorado
    "SD": 1049,  # South Dakota
    "NE": 7347,  # Nebraska
    "KS": 6961,  # Kansas
    "MN": 5633,  # Minnesota
    "IA": 32175,  # Iowa
    "MO": 41680,  # Missouri
    # Southwest region:
    "NM": 3536,  # New Mexico
    "TX": 80438,  # Texas
    "OK": 13517,  # Oklahoma
    "AR": 4910,  # Arkansas
    "LA": 15011,  # Louisiana
    # Midwest region:
    "WI": 37448,  # Wisconsin
    "IL": 72839,  # Illinois
    "MI": 105_181,  # Michigan
    "IN": 43994,  # Indiana
    "KY": 3870,  # Kentucky
    "OH": 155_123,  # Ohio
    # Northeast region:
    "ME": 15829,  # Maine
    "NH": 4546,  # New Hampshire
    "RI": 17000,  # Rhode Island
    "NJ": 21154,  # New Jersey
    "NY": 160_917,  # New York
    "PA": 65108,  # Pennsylvania
    "CT": 26_187,  # Connecticut
    "MA": 37087,  # Massachusetts
    "VA": 17667,  # Virginia
    "WV": 9168,  # West Virginia
    "MD": 19_284,  # Maryland
    "VT": 2_928,  # Vermont
    "DE": 3_044,  # Delaware
    # Southeast region:
    "TN": 42_479,  # Tennessee
    "MS": 15_205,  # Mississippi
    "AL": 15_835,  # Alabama
    "GA": 29_559,  # Georgia
    "FL": 46_405,  # Florida
    "SC": 5_680,  # South Carolina
    "NC": 28_348,  # North Carolina
}
S = state_demand  # Sales

print("Total states number: ", len(S))
print("Total demand: ", sum(S.values()))
print("Each state represents a demand center")

# NOTE: Valor total deve ser 1_477_026, então precisa ajustar NY (que nao tem na foto) pra bater o valor final.

In [ ]:
demand_centers = [f"{s}" for s in state_demand.keys()]

# pprint(demand_centers)

print(
    """
    The company has more than 70,000 individual customer accounts, and these are aggregated into 191 active demand centers.
    A demand center is a grouping of zip code areas into a zip sectional center as the focus of the collected demand.
    These demand centers, along with how they are currently being served, are given in Table 3
    """
)

In [ ]:
# 2.2 Plant current capacities C[p]
C = {
    "Covington": 620_000,
    "New York": 430_000,
    "Arlington": 300_000,
    "Long Beach": 280_000,
}

# Plant stocking capacities C'[p]
C_prime = {
    "Covington": 450_000,
    "New York": 380_000,
    "Arlington": 140_000,
    "Long Beach": 180_000,
}

In [ ]:
# 2.3 Unit production cost v[p] (variable production cost)
rho = {
    "Covington": 21.0,
    "New York": 19.9,
    "Arlington": 21.6,
    "Long Beach": 21.1,
}

In [ ]:
# 2.4 Distance matrices (in miles)
#     d_pw[p][w] = distance plant → warehouse
#     d_ws[w][d] = distance warehouse → demand center
#     d_pd[p][d] = distance plant → demand center

df_pw = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_pw", index_col=0
)
df_ws = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_ws", index_col=0
)
df_pd = pd.read_excel(
    "../distances/distance_matrix.xlsx", sheet_name="d_pd", index_col=0
)

d_pw = df_pw.to_dict()
d_ws = df_ws.to_dict()
d_pd = df_pd.to_dict()

In [ ]:
# print("Distance matrices:")
# print("d_pw (plant to warehouse):")
# pprint(d_pw)
# print("d_wd (warehouse to demand center):")
# pprint(d_ws)
# print("d_pd (plant to demand center):")
# pprint(d_pd)


In [ ]:
# 2.5 Inbound cost c_in[p][w] = 0.92 + 0.0034*d_pw
c_in = {
    p: {w: 0.92 + 0.0034 * d_pw[p][w.replace("-", "")] for w in warehouses}
    for p in plants
}

# c_in

In [ ]:
# Ler tabela 3 do excel
table3 = pd.read_excel(
    "../misc/tabelas.xlsx",
    sheet_name="table3",
)

# drop first 3 rows
# table3 = table3.drop(index=[0, 1, 2, 3])
# reset index
# table3 = table3.reset_index(drop=True)

table3.head()

In [ ]:
# for k, v in d_ws.items():
#     for k2, v2 in v.items():
#         if v2 <= 30:
#             print(k, k2, v2)

In [ ]:
# 2.6 Outbound cost c_out[w][j]:
local_rate = {i: v for i, v in enumerate(table3["Local delivery rate ($/cwt)"])}

c_out = {}
for i, w in enumerate(warehouses):
    w = w.replace("-", "")
    c_out[w] = {}
    for s in demand_centers:
        d = d_ws[w][s]
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            c_out[w][s] = local_rate[i]
        else:
            # else use 5.45 + 0.0037*d
            c_out[w][s] = 5.45 + 0.0037 * d

# c_out

In [ ]:
# pprint(c_out)

In [ ]:
# custos de transporte da fabrica direto para o demand center.
c_out_prime = {}

for p in plants:
    c_out_prime[p] = {}
    for j in demand_centers:
        try:
            d = d_pd[p][j]
        except KeyError:
            print(f"{p} or {j} not found in d_pd")
            break
        if d <= 30:
            # if d<=30: use local cartage rate from Table 3
            # c_out_prime[p][j] = local_rate[p]
            c_out_prime[p][j] = 0

            # TODO: verify this.
        else:
            # else use 5.45 + 0.0037*d
            c_out_prime[p][j] = 5.45 + 0.0037 * d

In [ ]:
# Quanto representa 1 cwt. em $? Dividimos as vendas totais ($) pela demand total (cwt)
Gamma = 160_000_000 / 1_477_026  # ($ / cwt)
Gamma

In [ ]:
# Custo de estocagem do armazém w
tau = {}

for i, w in enumerate(warehouses):
    # Fiz o casting para evitar o np.float64 nos prints, mas nao precisava
    tau[w] = float(table3["Storage ($/$)"][i])

In [ ]:
# Custo de handling do armazém w (epsilon)

epsilons = {}

for i, w in enumerate(warehouses):
    epsilons[w] = float(table3["Handling ($/cwt)"][i])

In [ ]:
# gamma

gammas = {}

for i, w in enumerate(warehouses):
    gammas[w] = float(table3["Stock Order Processing ($/order)"][i])

In [ ]:
# delta

deltas = {}

for i, w in enumerate(warehouses):
    deltas[w] = float(table3["Stock Order Size"][i])

In [ ]:
# omega

omegas = {}

for i, w in enumerate(warehouses):
    omegas[w] = float(table3["Customer Order size (cwt/order)"][i])

In [ ]:
# phi

phi = {}
for i, w in enumerate(warehouses):
    phi[w] = float(table3["Customer Order Processing ($/order)"][i])

## 3. Defining the model

In [ ]:
m = Model(name="UsemoreWarehousing")

### Indices

In [ ]:
print(plants)
print(warehouses)
# NOTE: tem que ver se o "New York-" não vai bugar quando permitirmos os potentials
print(demand_centers)

### Decision variables

In [ ]:
Z = m.addVars(warehouses, vtype=GRB.BINARY, name="z")
X = m.addVars(plants, warehouses, vtype=GRB.CONTINUOUS, name="x")
Y = m.addVars(warehouses, demand_centers, vtype=GRB.CONTINUOUS, name="y")

## 4. Objective Function

### Define all the costs components

In [ ]:
# inbound transport (from plants to warehouses) (OK)
inbound_transport_cost = gurobipy.quicksum(
    c_in[p][w] * X[p, w] for p in plants for w in warehouses if w != p
)

In [ ]:
# outbound transport (from warehouses to demand nodes) (OK)
outbound_transport_cost = gurobipy.quicksum(
    c_out[w.replace("-", "")][d] * Y[w.replace("-", ""), d]
    for w in existing_warehouses
    for d in demand_centers
)

In [ ]:
# transport cost from plants to demand centers (OK)
direct_transport_cost = gurobipy.quicksum(
    c_out_prime[p][d] * Y[p, d] for p in plants for d in demand_centers
)

In [ ]:
# production cost (OK)
production_costs1 = gurobipy.quicksum(
    rho[p] * (X[p, w]) for p in plants for w in warehouses
)
production_costs2 = gurobipy.quicksum(
    rho[p] * (Y[p, d]) for p in plants for d in demand_centers
)

In [ ]:
# storage cost in warehouses (OK)
m.Params.NonConvex = 2
totX = m.addVar(name="totX")

m.addConstr(
    totX == gurobipy.quicksum(X[p, w] for p in plants for w in warehouses),
    name="define_totX",
)

storage_pow = m.addVar(name="storage_pow")
m.addGenConstrPow(totX, storage_pow, 0.58, name="storage_pow")

storage_costs = 26 * 11.3 * storage_pow

In [ ]:
# INVENTORY (OK)
inventory_pow = m.addVar(name="inventory_pow")
m.addGenConstrPow(totX, inventory_pow, 0.58, name="inventory_pow")

inventory_carrying_costs = 26 * 0.12 * 11.3 * inventory_pow

In [ ]:
# custo de handling nos armazéns (OK)
handling_costs = gurobipy.quicksum(
    2 * Z[w] * X[p, w] * epsilons[w] for p in plants for w in warehouses
)

In [ ]:
# custo de processamento do pedido de estoque (OK)
stock_order_processing_costs = gurobipy.quicksum(
    Z[w] * X[p, w] * (gammas[w] / deltas[w]) for p in plants for w in warehouses
)

In [ ]:
# custo de processamento do pedido do cliente (OK)
customer_order_processing_costs = gurobipy.quicksum(
    Y[w, d] * (phi[w] / omegas[w]) * Z[w] for w in warehouses for d in demand_centers
)

#### Define the objective function

In [ ]:
m.setObjective(
    inbound_transport_cost
    + outbound_transport_cost
    + direct_transport_cost
    + production_costs1
    + production_costs2
    + storage_costs
    + handling_costs
    + stock_order_processing_costs
    + customer_order_processing_costs
    + inventory_carrying_costs,
    sense=GRB.MINIMIZE,
)

## Constraints

In [ ]:
# Nível de serviço
# tabela_4 = {
#     100: 56.4,
#     200: 77.7,
#     300: 93.4,
#     400: 95.5,
#     500: 97.0,
#     600: 97.5,
#     700: 99.5,
#     800: 100.0,
# }


# def get_valor_tabela_4(distancia):
#     for k, v in tabela_4.items():
#         if distancia <= k:
#             return v
#     return 100.0


# * get_valor_tabela_4(d_ws[w][d]) / 100



In [ ]:
# Porcentagem de volume atendidos a uma distância menor que SERVICE_LEVEL_DISTANCE milhas
total_prod = sum(S.values())

service_level = gurobipy.quicksum(
    Y[w, d] / total_prod
    for w in warehouses
    for d in demand_centers
    if d_ws[w][d] <= SERVICE_LEVEL_DISTANCE
)


# m.addConstr(
#     service_level >= SERVICE_LEVEL,
#     name="service_level",
# )

In [ ]:
import numpy as np


# Distancias medianas de um armazém para os centros de demanda

for w in d_ws.keys():
    q1 = np.percentile(list(d_ws[w].values()), 95)
    # print(f"{w:>20} {q1:>7.2f}")

In [ ]:
# Demand satisfaction (OK)

for d in demand_centers:
    m.addConstr(
        gurobipy.quicksum(Y[w, d] for w in warehouses) == S[d],
        name=f"demand_{d}",
    )

In [ ]:
# Plant production capacity (OK)

for p in plants:
    # NOTE: repare que o que fica na fábrica pra ir direto pra d já está incluso no X[p, p]
    m.addConstr(
        gurobipy.quicksum(X[p, w] for w in warehouses) <= C[p],
        name=f"plantCap_{p}",
    )

In [ ]:
# restrição dos 10400 (OK)

for w in warehouses:
    # Um armazém só pode ser aberto se houver demanda de pelo menos 10_400 cwt
    m.addConstr(
        gurobipy.quicksum(Y[w, d] for d in demand_centers) >= 10_400 * Z[w],
        name=f"minThroughput_{w}",
    )

In [ ]:
# restrições de armazenagem de cada planta (OK)

for p in plants:
    m.addConstr(X[p, p] <= C_prime[p])

In [ ]:
# Flow balance at each warehouse (está quebrando o código...)

# for w in warehouses:
#     # Tudo que entra no armazém deve ser igual a tudo que sai
#     m.addConstr(
#         gurobipy.quicksum(X[p, w] for p in plants)
#         == gurobipy.quicksum(Y[w, d] for d in demand_centers),
#         name=f"flowBalance_{w}",
#     )

# só para depósitos reais:
for w in existing_warehouses:
    m.addConstr(
        gurobipy.quicksum(X[p, w] for p in plants)
        == gurobipy.quicksum(Y[w, d] for d in demand_centers),
        name=f"flowExternal_{w}",
    )

# para cada planta-nó:
for p in plants:
    # tudo que sai de p, seja via X (para outros warehouses) ou via Y (direto),
    # deve bater com tudo que entra em p (X[p,p], se você ainda quiser usá-lo como estoque interno)
    m.addConstr(
        # opcional, se mantiver X[p,p] como "estoque p"
        X[p, p] + gurobipy.quicksum(Y[p, d] for d in demand_centers)
        == gurobipy.quicksum(X[n, p] for n in plants),
        name=f"flowPlant_{p}",
    )

In [ ]:
M = total_prod

In [ ]:
# Restrições de Z (só posso usar um armazém se ele estiver aberto) (OK)
for w in warehouses:
    for p in plants:
        # NOTE: the C[p] here is working as the big-M
        m.addConstr(
            X[p, w] <= M * Z[w],
            name=f"openWarehouse_{w},{p}",
        )
    for d in demand_centers:
        # NOTE: the S[d] here is working as the big-M
        m.addConstr(
            Y[w, d] <= M * Z[w],
            name=f"openWarehouse_{w},{d}",
        )

In [ ]:
# NOTE: forçando a planta a nao enviar direto pro armazém
# for p in plants:
#     for d in demand_centers:
#         if d_pd[p][d] >= 300:
#             continue

#         m.addConstr(
#             Y[p, d] == 0,
#             name=f"maluco_{p},{d}",
#         )


In [ ]:
# Cada cliente deve ser atendido por um warehouse que esteja no máximo a 300 milhas de distancia
# for w in existing_warehouses + plants:
#     for d in demand_centers:
#         if d_ws[w][d] > 900:
#             m.addConstr(
#                 Y[w, d] == 0,
#                 name=f"dist_too_far_{w}_{d}"
#             )


In [ ]:
# pprint(d_ws)

# import numpy as np

# np.mean(
#     [d_ws[w][d] for w in existing_warehouses + plants for d in demand_centers]
# )


## Solver

In [ ]:
# NOTE: precisou ser o formato .rlp pois algumas variáveis têm espaço
m.write("model-baseline.rlp")

In [ ]:
m.params.TimeLimit = 300  # seconds

m.optimize()

In [ ]:
m.printStats()

In [ ]:
m.printQuality()

## Visualize results

### Costs

In [ ]:
# Print the solution objective value
print(f"Objective function final value: ${m.objVal:,.2f}")

In [ ]:
def display_cost_components():
    """Calculate and display all cost components"""

    # Get cost values from the model
    costs = {
        "Inbound transport cost": inbound_transport_cost.getValue(),
        "Outbound transport cost": outbound_transport_cost.getValue(),
        "Direct transport cost": direct_transport_cost.getValue(),
        # "Local transport cost": local_transport.getValue(),
        # "Long transport cost": long_transport.getValue(),
        "Production costs": production_costs1.getValue() + production_costs2.getValue(),
        "Storage costs": storage_costs.getValue(),
        "Handling costs": handling_costs.getValue(),
        "Stock order processing costs": stock_order_processing_costs.getValue(),
        "Customer order processing costs": customer_order_processing_costs.getValue(),
        "Inventory Carrying Costs": inventory_carrying_costs.getValue(),
    }

    # Calculate total cost
    total = sum(costs.values())

    # Print the cost components in a formatted table
    print("\n" + "=" * 65)
    print(f"{'COST BREAKDOWN':^65}")
    print("=" * 65)
    print(f"{'Cost Component':<30} {'Amount ($)':>18} {'Percentage':>12}")
    print("-" * 65)

    try:
        for name, value in costs.items():
            percentage = (value / total) * 100
            print(f"{name:<32} ${value:>17,.2f} {percentage:>11.2f}%")
    except ZeroDivisionError:
        pass

    print("-" * 65)
    print(f"{'TOTAL COST':<32} ${total:>17,.2f} {100:>11.2f}%")
    print("=" * 65)


# Call the function to display costs
display_cost_components()

### Locations

In [ ]:
open_w = [w for w in warehouses if Z[w].X > 0.1]
closed_w = [w for w in warehouses if Z[w].X < 0.1]

print(f"Number of open warehouses 🏭: {len(open_w)} ")
print(f"Number of closed warehouses 🔒: {len(closed_w)} ")


# Calculate percentage of open warehouses
percentage_open = (len(open_w) / len(warehouses)) * 100
print(f"Percentage of warehouses open: {percentage_open:.1f}% 📊")

print("\n--- Warehouse openings ---")
for w in open_w:
    print(f" Open warehouse at {w}")

if len(closed_w) > 0:
    print("\n--- Warehouse closings ---")
    for w in closed_w:
        print(f" Closed warehouse at {w}")

In [ ]:
service_level.getValue()

In [ ]:
# TODO: Salvar os open warehouses em um arquivo


### Production

In [ ]:
plant_production = {}

for p in plants:
    plant_production[p] = sum(X[p, w].X for w in warehouses if Z[w].X > 0.1) + sum(
        Y[p, d].X for d in demand_centers if d in S and S[d] > 0 and Y[p, d].X > 0.1
    )

print("\n------------------ Plant production ----------------------")
total_production = sum(plant_production.values())
for p in plants:
    percentage = (plant_production[p] / total_production) * 100
    print(
        f" Plant {p:>15} production: {plant_production[p]:,.2f} cwt ({percentage:>4.1f}%)"
    )

In [ ]:
print(f"Total production: {total_production:,.1f} cwt")

### Flows

In [ ]:
print("\n---------- Plant → Warehouse flows -------------")
for p, w in X.keys():
    if X[p, w].X > 1e-6:
        print(f" {p:>15} → {w:<16} = {X[p, w].X:>5.0f} cwt")

In [ ]:
print("\n---------- Plant → Demand flows ---")
for w, j in Y.keys():
    if Y[w, j].X > 1e-6:
        print(f" {w:>15} → {j}: {Y[w, j].X:>7.0f} cwt")

In [ ]:
print("\n------- Warehouse → Demand flows ---")
for w, j in Y.keys():
    if Y[w, j].X > 1e-6:
        print(f" {w:>16} → {j}: {Y[w, j].X:>5.0f} cwt")